**Jonathan Ng**

**Vinty Dong**

**CSE 354 Final Project Notebook 3**

This note book was used in order to automatically evaluate Flan-T5 models of various sizes on the math_qa dataset from huggingface using few-shot and zero-shot prompting. 

In [10]:
!pip install transformers
!pip install datasets

In [1]:
from datasets import load_dataset
import torch

dataset = load_dataset("math_qa")

c:\Users\waibo\miniconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset math_qa (C:/Users/waibo/.cache/huggingface/datasets/math_qa/default/0.1.0/67fc1cc5d22b185002c6fd16e19e4d5215eae01fb04d656bed83204ba6ee55ff)
100%|██████████| 3/3 [00:00<00:00, 41.10it/s]


In [2]:
train_data = dataset['train']
test_data = dataset['test']
validate_data = dataset['validation']

print(len(train_data))
print(len(test_data))
print(len(validate_data))

29837
2985
4475


In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from collections import defaultdict

In [11]:
# inputs = tokenizer("average age of students of an adult school is 40 years . 120 new students whose average age is 32 years joined the school . as a result the average age is decreased by 4 years . find the number of students of the school after joining of the new students .", return_tensors="pt")
# outputs = model.generate(**inputs)
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
# # ['Pour a cup of bolognese into a large bowl and add the pasta']

def evaluate_flan_T5(FEW_SHOT, CHOOSE_BEST_OPTION, name = 'google/flan-t5-large'):

  model = AutoModelForSeq2SeqLM.from_pretrained(name)
  tokenizer = AutoTokenizer.from_pretrained(name)
  results = {}
  counts = {}
  non_single = 0

  correct = defaultdict(list)
  incorrect = defaultdict(list)

  c = 100

  # Old Prompt = 'Q: [] A: []' ~ 10%
  # Current Prompt = "Q: [] Choose the best option: []" ~ 24%

  test_length = len(test_data)
  print("Total Samples", test_length)

  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model = model.to(device)



  # Generate k samples for few-shot
  # Specifically avoid the ith sample
  def get_examples(i, k=3):
    s = ''
    while k > 0:
      r = torch.randint(low=0, high=test_length, size=(1,))
      if(r == i):
        continue
      s = s + '\nQ: ' + test_data[r]['Problem'][0] + test_data[r]['correct'][0]
      k = k - 1
    return s

  FEW_SHOT = True
  #FEW_SHOT = False

  if FEW_SHOT:
    print("Few Shot", name)

  for i, problem in enumerate(test_data):
    prompt = ''

    if FEW_SHOT:
      prompt = get_examples(i, k=3)
      prompt = prompt + '\n'

    if CHOOSE_BEST_OPTION:
      prompt = prompt + 'Q: ' + problem['Problem'] + ' Choose the best option: ' + problem['options']
    else:
      prompt = prompt + 'Q: ' + problem['Problem'] + ' A: ' + problem['options']

    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = inputs.to(device)
    outputs = model.generate(**inputs).to(device)
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    category = problem['category']

    if(len(res) != 1):
      non_single = non_single + 1

    if(res == problem['correct']):
      results[category] = results.get(category, 0) + 1
      correct[category].append(f"{problem['Problem']}")
    else:
      incorrect[category].append(f"{problem['Problem']}\n {problem['options']} Correct Reponse: {problem['correct']} Models Incorrect Response: {res}")

    counts[category] = counts.get(category, 0) + 1

    # print(prompt)
    # print(res)
    # break
    if(i % 100 == 0):
      print(f"{i}th Testing Sample")

    # c = c - 1
    # if(c < 0):
    #   break
  print(counts)
  print(results)
  print(non_single, " Non single responess")
  return correct, incorrect
  





In [12]:
correct, incorrect = evaluate_flan_T5(FEW_SHOT=True, CHOOSE_BEST_OPTION=True, name='google/flan-t5-large')

Total Samples 2985
Few Shot google/flan-t5-large


c:\Users\waibo\miniconda3\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


0th Testing Sample
100th Testing Sample
200th Testing Sample
300th Testing Sample
400th Testing Sample
500th Testing Sample
600th Testing Sample
700th Testing Sample
800th Testing Sample
900th Testing Sample
1000th Testing Sample
1100th Testing Sample
1200th Testing Sample
1300th Testing Sample
1400th Testing Sample
1500th Testing Sample
1600th Testing Sample
1700th Testing Sample
1800th Testing Sample
1900th Testing Sample
2000th Testing Sample
2100th Testing Sample
2200th Testing Sample
2300th Testing Sample
2400th Testing Sample
2500th Testing Sample
2600th Testing Sample
2700th Testing Sample
2800th Testing Sample
2900th Testing Sample
{'gain': 558, 'physics': 702, 'other': 172, 'general': 1301, 'geometry': 200, 'probability': 52}
{'physics': 242, 'other': 52, 'general': 379, 'gain': 170, 'geometry': 52, 'probability': 12}
0  Non single responess


In [13]:

for category, problems in correct.items():
    print(f"{category}: Correct problems")
    print("\n")
    for problem in problems:
        print(problem)
        print("\n")

print("\n")

for category, problems in incorrect.items():
    print(f"{category}: Incorrect Problems")
    print("\n")
    for problem in problems:
        print(problem)
        print("\n")


physics: Correct problems


there are 28 stations between hyderabad and bangalore . how many second class tickets have to be printed , so that a passenger can travel from any station to any other station ?


a constructor estimates that 10 people can paint mr khans house in 4 days . if he uses 5 people instead of 10 , how long will they take to complete the job ?


a train 300 m long is running at a speed of 45 km / hr . in what time will it pass a bridge 150 m long ?


a train running at the speed of 120 km / hr crosses a pole in 18 seconds . what is the length of the train ?


the lenght of a room is 5.5 m and width is 4 m . find the cost of paving the floor by slabs at the rate of rs . 900 per sq . metre .


a start walking from a place at a uniform speed of 6 kmph in a particular direction . after half an hour , b starts from the same place and walks in the same direction as a at a uniform speed and overtakes a after 1 hour 48 minutes . find the speed of b .


the cross - section o